<a href="https://colab.research.google.com/github/pranjall702/Wallet-Risk-Scoring-From-Scratch/blob/main/wallet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import files
uploaded = files.upload()


Saving Wallet id (1).xlsx to Wallet id (1) (1).xlsx


In [8]:
!pip install gql requests pandas openpyxl


In [9]:
import pandas as pd
import requests
from tqdm import tqdm


In [11]:
# Load Excel file (use the correct uploaded filename)
wallets_df = pd.read_excel("/content/Wallet id (1).xlsx")
wallets = wallets_df.iloc[:, 0].dropna().unique().tolist()
wallets[:5]  # Preview


['0x0039f22efb07a647557c7c5d17854cfd6d489ef3',
 '0x06b51c6882b27cb05e712185531c1f74996dd988',
 '0x0795732aacc448030ef374374eaae57d2965c16c',
 '0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9',
 '0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae']

In [12]:
COMPOUND_GRAPH_URL = "https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2"

def fetch_compound_data(wallet_id):
    query = """
    query ($user: ID!) {
      account(id: $user) {
        id
        tokens {
          symbol
          supplyBalanceUnderlying
          borrowBalanceUnderlying
        }
      }
    }
    """
    try:
        response = requests.post(
            COMPOUND_GRAPH_URL,
            json={"query": query, "variables": {"user": wallet_id.lower()}}
        )
        result = response.json()["data"]["account"]
        return result
    except:
        return None


In [13]:
def calculate_risk_score(wallet_data):
    if wallet_data is None:
        return 1000  # max risk if no data

    supplied = 0
    borrowed = 0

    for token in wallet_data.get("tokens", []):
        s = float(token.get("supplyBalanceUnderlying", 0))
        b = float(token.get("borrowBalanceUnderlying", 0))
        supplied += s
        borrowed += b

    # Risk Score Logic: higher borrow relative to supply = higher risk
    if supplied == 0 and borrowed == 0:
        return 1000  # inactive wallets = risky
    elif borrowed == 0:
        return 100  # safe
    elif supplied == 0:
        return 900  # very risky
    else:
        ratio = borrowed / (supplied + 1e-6)
        score = min(int(ratio * 1000), 1000)
        return score


In [14]:
output = []

for wallet in tqdm(wallets):
    data = fetch_compound_data(wallet)
    score = calculate_risk_score(data)
    output.append({"wallet_id": wallet, "score": score})

result_df = pd.DataFrame(output)
result_df.to_csv("wallet_scores.csv", index=False)
print("✅ wallet_scores.csv generated")


100%|██████████| 103/103 [00:07<00:00, 14.14it/s]

✅ wallet_scores.csv generated


In [15]:
files.download("wallet_scores.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>